In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = ''
# add outcomes
outcomes=pd.read_csv(path+'PD_Outcome.csv')[['PATNO','EVENT_ID', 'INFODT', 'HY']]
outcomes['INFODT'] = pd.to_datetime(outcomes['INFODT'])
print('original len:',len(outcomes))
# add tabular features
pd_patients=pd.read_csv(path+'pd_on.csv')
pd_patients=pd_patients.replace({'UR': pd.NA})
# basic features
demo = pd.read_csv(path+'demographics.csv')
# medication
ledd = pd.read_csv(path+'ledd_new.csv')
# gene 
gene = pd.read_csv(path+'genes.csv')
mri = pd.read_csv(path+'mri_segments.csv')
# combine basic and outcomes
outcomes=outcomes.merge(demo[['PATNO', 'ENROLL_AGE', 'SEX']], on='PATNO', how= 'inner')
outcomes=outcomes.replace({'UR': pd.NA})
print('after merging len:',len(outcomes))

# combine tarbular and medication information
pd_patients = pd_patients.merge(ledd, on = ['PATNO','EVENT_ID', 'INFODT'], how ='inner')
hc = pd.read_csv(path+'hc.csv')
len(pd_patients)

In [ ]:
# Only select BL, M12, M24 and M36 timeslots for features!!!
target_evnents =['BL', 'SC', 'V04', 'V06', 'V08'] # select 12 month(we do not need features from 12 month) 
bs_patients =pd_patients[pd_patients['EVENT_ID'].isin(target_evnents)]
print(len(bs_patients))
bs_patients.drop_duplicates(subset=['PATNO', 'EVENT_ID', 'INFODT'], keep='last', inplace=True)
print(len(bs_patients))

In [ ]:
bs_hc =hc[hc['EVENT_ID'].isin(target_evnents)]
print(len(bs_hc))
bs_hc.drop_duplicates(subset=['PATNO', 'EVENT_ID', 'INFODT'], keep='last', inplace=True)
print(len(bs_hc))

In [ ]:
# merge SC and BL features
def sc_bl_imputation(bs_patients):
    uni_idx = set(bs_patients['PATNO'])
    BL_event = ['BL', 'SC']
    other_event =  ['V04', 'V06', 'V08',]
    new_bs_patients=pd.DataFrame(columns=bs_patients.columns)
    counts = []
    for idx in uni_idx:
        #for BL and SC
        df = bs_patients[bs_patients['PATNO']==idx]
        tar_df = df[df['EVENT_ID'].isin(BL_event)]
        tar_df['INFODT']= pd.to_datetime(tar_df['INFODT'])
        tar_df.sort_values(by='INFODT',ascending=True,inplace=True)
        tar_df = tar_df.fillna(method='ffill') # use SC information to fill BL
        tar_df = tar_df.fillna(method='bfill') # use SC information to fill BL
        # for other evetnts
        other_tar = df[df['EVENT_ID'].isin(other_event)]
        new_bs_patients = pd.concat([new_bs_patients,tar_df])
        new_bs_patients = pd.concat([new_bs_patients,other_tar])
        counts.append(len(tar_df))
    print(len(new_bs_patients))
    new_bs_patients = new_bs_patients[new_bs_patients.EVENT_ID.isin(['BL','V04', 'V06', 'V08'])]
    print(len(new_bs_patients))
    return new_bs_patients

new_bs_patients = sc_bl_imputation(bs_patients)
new_bs_hc = sc_bl_imputation(bs_hc)


In [ ]:
# before data inputation, we need to check the label missing rate
def missing_check(new_bs_patients, merge = True):
    middle_tar=['NP1RTOT','NP1PTOT','NP2PTOT','NP3TOT', 'NHY','MoCA_score','ESS_TOT']
    bs_patients_tar = new_bs_patients[middle_tar]
    bs_patients_tar['label_missing'] = bs_patients_tar.isna().sum(axis=1)
    bs_patients_tar['label_missing'].value_counts()
    bs_patients_tar = bs_patients_tar[bs_patients_tar['label_missing']<=3]
    new_bs_patients = new_bs_patients[new_bs_patients.index.isin(bs_patients_tar.index)]
    # print(len(new_bs_patients))
    # adding sex and age
    if merge:
        new_bs_patients = new_bs_patients.merge(outcomes[['PATNO', 'EVENT_ID', 'ENROLL_AGE', 'SEX']], on=['PATNO', 'EVENT_ID'], how='inner')
    new_bs_patients.drop_duplicates(inplace=True)
    print(len(new_bs_patients))
    return new_bs_patients

new_bs_patients = missing_check(new_bs_patients)

new_bs_hc = missing_check(new_bs_hc, False)
new_bs_hc = new_bs_hc[new_bs_hc['NHY']==str(0)]


## select features and labels 

In [ ]:
interest_cols = [
'PATNO',
'EVENT_ID',
'ENROLL_AGE',
'SEX',
'INFODT',
'NP1COG',
'NP1HALL',
'NP1DPRS',
'NP1ANXS',
'NP1APAT',
'NP1DDS',
'NP1SLPN',
'NP1SLPD',
'NP1PAIN',
'NP1URIN',
'NP1CNST',
'NP1LTHD', # U1 scores
'NP1FATG',
'NP2SPCH',
'NP2SALV',
'NP2SWAL',
'NP2EAT',
'NP2DRES',
'NP2HYGN',
'NP2HWRT',
'NP2HOBB',
'NP2TURN',
'NP2TRMR',
'NP2RISE',
'NP2WALK',
'NP2FREZ', # u2 scores
'MSEADLG', # Modifier S and E overall score
'NHY', # U3 3.21
'NP3SPCH',
'NP3FACXP',
'NP3RIGN',
'NP3RIGRU',
'NP3RIGLU',
'NP3RIGRL',
'NP3RIGLL',
'NP3FTAPR',
'NP3FTAPL',
'NP3HMOVR',
'NP3HMOVL',
'NP3PRSPR',
'NP3PRSPL',
'NP3TTAPR',
'NP3TTAPL',
'NP3LGAGR',
'NP3LGAGL',
'NP3RISNG',
'NP3GAIT',
'NP3FRZGT',
'NP3PSTBL',
'NP3POSTR',
'NP3BRADY',
'NP3PTRMR',
'NP3PTRML',
'NP3KTRMR',
'NP3KTRML',
'NP3RTARU',
'NP3RTALU',
'NP3RTARL',
'NP3RTALL',
'NP3RTALJ',
'NP3RTCON',
'NP3TOT',
'SDMTOTAL', # Symbol Digit Modalities,  Neuropsychological Test, tot
'STAI_TOT', # State-Trait Anxiety Inventory for Adults tot 
'SFT_TOT', # Semantic Fluency tot
'SCOPA_AUT_TOT', #SCOPA-AUT, autonomic test, Lower score means better.
'REMSLEEP_TOT', # REM Sleep Behavior Questionnaire
'QUIP_A', # This is a questionnaire about gambling, buying, etc. It is a nerobehavioral questionnaire.
'QUIP_B',
'QUIP_C',
'QUIP_D',
'QUIP_E',
# 'UPSIT_TOT', # University_of_Pennsylvania_Smell_Identification_Test__UPSIT
'MoCA_score',
'LNS_TOT', # Letter Number Sequencing, total score
'HVLT_TOT_Recall', # Hopkins Verbal Learning Test， HVLT Immediate/Total Recall	Sum of HVLTRT1 - HVLTRT3
'HVLT_DCR_REC',
'HVLT_RETENTION',
'GDS_TOT', # The Geriatric Depression Scale (GDS) is a 30-item self-report assessment used to identify depression 
'GDS_Depressed', # if GDS tot >5
'ESS_TOT', # Epworth Sleepiness Scale
'ESS_Sleepy', # if ESS_ToT>10
'BJLOT_TOT', #  Benton Judgement of Line Orientation total score
'DATSCAN_CAUDATE_R', # 'CAUDATE_R',
'DATSCAN_CAUDATE_L', # 'CAUDATE_L',
'DATSCAN_PUTAMEN_R',# 'PUTAMEN_R',
'DATSCAN_PUTAMEN_L',# 'PUTAMEN_L']
# analysis tagets
'NP1RTOT',
'NP1PTOT',
'NP2PTOT',
'LEDD'
]

In [ ]:
print(len(interest_cols))
print(len(set(interest_cols)))

In [ ]:
new_feature_cols = [
'PATNO',
'EVENT_ID',
'ENROLL_AGE',
'SEX',
'INFODT',
'NP1COG',
'NP1HALL',
'NP1DPRS',
'NP1ANXS',
'NP1APAT',
'NP1DDS',
'NP1SLPN',
'NP1SLPD',
'NP1PAIN',
'NP1URIN',
'NP1CNST',
'NP1LTHD', # U1 scores
'NP1FATG',
'NP2SPCH',
'NP2SALV',
'NP2SWAL',
'NP2EAT',
'NP2DRES',
'NP2HYGN',
'NP2HWRT',
'NP2HOBB',
'NP2TURN',
'NP2TRMR',
'NP2RISE',
'NP2WALK',
'NP2FREZ', # u2 scores
'MSEADLG', # Modifier S and E overall score
'NHY', # U3 3.21
'NP3SPCH',
'NP3FACXP',
'NP3RIGN',
'NP3RIGRU',
'NP3RIGLU',
'NP3RIGRL',
'NP3RIGLL',
'NP3FTAPR',
'NP3FTAPL',
'NP3HMOVR',
'NP3HMOVL',
'NP3PRSPR',
'NP3PRSPL',
'NP3TTAPR',
'NP3TTAPL',
'NP3LGAGR',
'NP3LGAGL',
'NP3RISNG',
'NP3GAIT',
'NP3FRZGT',
'NP3PSTBL',
'NP3POSTR',
'NP3BRADY',
'NP3PTRMR',
'NP3PTRML',
'NP3KTRMR',
'NP3KTRML',
'NP3RTARU',
'NP3RTALU',
'NP3RTARL',
'NP3RTALL',
'NP3RTALJ',
'NP3RTCON',
'NP3TOT',
'SDMTOTAL', # Symbol Digit Modalities,  Neuropsychological Test, tot
'STAI_TOT', # State-Trait Anxiety Inventory for Adults tot 
'SFT_TOT', # Semantic Fluency tot
'SCOPA_AUT_TOT', #SCOPA-AUT, autonomic test, Lower score means better.
'REMSLEEP_TOT', # REM Sleep Behavior Questionnaire
'QUIP_A', # This is a questionnaire about gambling, buying, etc. It is a nerobehavioral questionnaire.
'QUIP_B',
'QUIP_C',
'QUIP_D',
'QUIP_E',
# 'UPSIT_TOT', # University_of_Pennsylvania_Smell_Identification_Test__UPSIT
'MoCA_score',
'LNS_TOT', # Letter Number Sequencing, total score
'HVLT_TOT_Recall', # Hopkins Verbal Learning Test， HVLT Immediate/Total Recall	Sum of HVLTRT1 - HVLTRT3
'HVLT_DCR_REC',
'HVLT_RETENTION',
'GDS_TOT', # The Geriatric Depression Scale (GDS) is a 30-item self-report assessment used to identify depression 
'GDS_Depressed', # if GDS tot >5
'ESS_TOT', # Epworth Sleepiness Scale
'ESS_Sleepy', # if ESS_ToT>10
'BJLOT_TOT', #  Benton Judgement of Line Orientation total score
'DATSCAN_CAUDATE_R', # 'CAUDATE_R',
'DATSCAN_CAUDATE_L', # 'CAUDATE_L',
'DATSCAN_PUTAMEN_R',# 'PUTAMEN_R',
'DATSCAN_PUTAMEN_L',# 'PUTAMEN_L']
# analysis tagets
'NP3_Akinetic_Rigid',
'NP3_Axial',
'NP3_Tremor',
'NP1TOT',
'NP2PTOT',
'NPTOT',
'LEDD'
]

In [ ]:
feature_cols = [
'NP1COG',
'NP1HALL',
'NP1DPRS',
'NP1ANXS',
'NP1APAT',
'NP1DDS',
'NP1SLPN',
'NP1SLPD',
'NP1PAIN',
'NP1URIN',
'NP1CNST',
'NP1LTHD', # U1 scores
'NP1FATG',
'NP2SPCH',
'NP2SALV',
'NP2SWAL',
'NP2EAT',
'NP2DRES',
'NP2HYGN',
'NP2HWRT',
'NP2HOBB',
'NP2TURN',
'NP2TRMR',
'NP2RISE',
'NP2WALK',
'NP2FREZ', # u2 scores
'MSEADLG', # Modifier S and E overall score
'NHY', # U3 3.21
'NP3SPCH',
'NP3FACXP',
'NP3RIGN',
'NP3RIGRU',
'NP3RIGLU',
'NP3RIGRL',
'NP3RIGLL',
'NP3FTAPR',
'NP3FTAPL',
'NP3HMOVR',
'NP3HMOVL',
'NP3PRSPR',
'NP3PRSPL',
'NP3TTAPR',
'NP3TTAPL',
'NP3LGAGR',
'NP3LGAGL',
'NP3RISNG',
'NP3GAIT',
'NP3FRZGT',
'NP3PSTBL',
'NP3POSTR',
'NP3BRADY',
'NP3PTRMR',
'NP3PTRML',
'NP3KTRMR',
'NP3KTRML',
'NP3RTARU',
'NP3RTALU',
'NP3RTARL',
'NP3RTALL',
'NP3RTALJ',
'NP3RTCON',
'NP3TOT',
'SDMTOTAL', # Symbol Digit Modalities,  Neuropsychological Test, tot
'STAI_TOT', # State-Trait Anxiety Inventory for Adults tot 
'SFT_TOT', # Semantic Fluency tot
'SCOPA_AUT_TOT', #SCOPA-AUT, autonomic test, Lower score means better.
'REMSLEEP_TOT', # REM Sleep Behavior Questionnaire
'QUIP_A', # This is a questionnaire about gambling, buying, etc. It is a nerobehavioral questionnaire.
'QUIP_B',
'QUIP_C',
'QUIP_D',
'QUIP_E',
# 'UPSIT_TOT', # University_of_Pennsylvania_Smell_Identification_Test__UPSIT
'MoCA_score',
'LNS_TOT', # Letter Number Sequencing, total score
'HVLT_TOT_Recall', # Hopkins Verbal Learning Test， HVLT Immediate/Total Recall	Sum of HVLTRT1 - HVLTRT3
'HVLT_DCR_REC',
'HVLT_RETENTION',
'GDS_TOT', # The Geriatric Depression Scale (GDS) is a 30-item self-report assessment used to identify depression 
'GDS_Depressed', # if GDS tot >5
'ESS_TOT', # Epworth Sleepiness Scale
'ESS_Sleepy', # if ESS_ToT>10
'BJLOT_TOT', #  Benton Judgement of Line Orientation total score
'DATSCAN_CAUDATE_R', # 'CAUDATE_R',
'DATSCAN_CAUDATE_L', # 'CAUDATE_L',
'DATSCAN_PUTAMEN_R',# 'PUTAMEN_R',
'DATSCAN_PUTAMEN_L',# 'PUTAMEN_L']
# analysis tagets
'NP3_Akinetic_Rigid',
'NP3_Axial',
'NP3_Tremor',
'NP1TOT',
'NP2PTOT',
'NPTOT',
'LEDD',
'PRS90',
'GBA',
'LRRK2',
'SNCA',
'APOE_E4'
]

In [ ]:
tar=[
'NP1TOT',
'NP2PTOT',
'NP3TOT',    
'NPTOT',
'NP3_Axial',
'NP3_Tremor',
'NP3_Akinetic_Rigid',
'NHY',
'MoCA_score',
'ESS_TOT'
]

reg_tar=[
'NP1TOT',
'NP2PTOT',
'NP3TOT',    
'NPTOT',
'NP3_Axial',
'NP3_Tremor',
'NP3_Akinetic_Rigid',
'MoCA_score',
'ESS_TOT'
]

In [ ]:
# fillna with mean data
def mean_fill(df, fill=False):
    mr = []
    clos = []
    new_df = pd.DataFrame()
    for column in interest_cols:
        if column not in ['PATNO', 'EVENT_ID', 'ENROLL_AGE', 'SEX','INFODT', 'LEDD']:
            missing_rate = df[column].isnull().sum()/len(df)
            mr.append(missing_rate)
            clos.append(column)
            new_df[column] = pd.to_numeric(df[column])
            if fill:
                mean_val = int(df[column].mean())
                df[column].fillna(mean_val, inplace=True)
        else:
            new_df[column] = df[column]
    rec_df = pd.DataFrame()
    rec_df['feature'] = clos
    rec_df['missing'] = mr
    return new_df, rec_df

In [ ]:
# fillna with mean data
def mean_fill_hc(df, fill=False):
    mr = []
    clos = []
    for column in interest_cols:
        if column not in ['PATNO', 'EVENT_ID', 'ENROLL_AGE', 'SEX','INFODT', 'LEDD']:
            missing_rate = df[column].isnull().sum()/len(df)
            mr.append(missing_rate)
            clos.append(column)
            df[column] = pd.to_numeric(df[column])
            if fill:
                mean_val = int(df[column].mean())
                df[column].fillna(mean_val, inplace=True)
    rec_df = pd.DataFrame()
    rec_df['feature'] = clos
    rec_df['missing'] = mr
    return df, rec_df

In [ ]:
bs_patients_new, missing_pa = mean_fill(new_bs_patients)
bs_hc_new, missing_hc = mean_fill_hc(new_bs_hc)

In [ ]:

def add_new_target(bs_patients_new):
    bs_patients_new['NP1TOT'] = bs_patients_new['NP1RTOT'].astype(float)+bs_patients_new['NP1PTOT'].astype(float)
    bs_patients_new['NPTOT'] = bs_patients_new['NP1RTOT'].astype(float)+bs_patients_new['NP1PTOT'].astype(float)+ bs_patients_new['NP2PTOT'].astype(float)+bs_patients_new['NP3TOT'].astype(float)
    # jiangzhi
    bs_patients_new['NP3_Rigidity']=bs_patients_new['NP3RIGLL'].astype(float)+bs_patients_new['NP3RIGLU'].astype(float)+bs_patients_new['NP3RIGN'].astype(float)+bs_patients_new['NP3RIGRL'].astype(float)+bs_patients_new['NP3RIGRU'].astype(float)
    # zhenchan
    bs_patients_new['NP3_RestTremer'] = bs_patients_new['NP3RTALJ'].astype(float)+bs_patients_new['NP3RTALL'].astype(float)+bs_patients_new['NP3RTALU'].astype(float)+bs_patients_new['NP3RTARL'].astype(float)+bs_patients_new['NP3RTARU'].astype(float)
    bs_patients_new['NP3_Axial'] =bs_patients_new['NP3RISNG'].astype(float)+bs_patients_new['NP3GAIT'].astype(float)+bs_patients_new['NP3FRZGT'].astype(float)+bs_patients_new['NP3PSTBL'].astype(float)+bs_patients_new['NP3POSTR'].astype(float)+bs_patients_new['NP3BRADY'].astype(float)
    bs_patients_new['NP3_Tremor']=bs_patients_new['NP3PTRMR'].astype(float)+bs_patients_new['NP3PTRML'].astype(float)+bs_patients_new['NP3KTRML'].astype(float)+bs_patients_new['NP3KTRMR'].astype(float)+bs_patients_new['NP3_RestTremer'].astype(float)+bs_patients_new['NP3RTCON'].astype(float)
    bs_patients_new['NP3_Akinetic_Rigid'] = bs_patients_new['NP3FACXP'].astype(float)+bs_patients_new['NP3_Rigidity']+bs_patients_new['NP3FTAPR'].astype(float)+bs_patients_new['NP3FTAPL'].astype(float)+bs_patients_new['NP3HMOVR'].astype(float)+bs_patients_new['NP3HMOVL'].astype(float)+bs_patients_new['NP3PRSPL'].astype(float)+bs_patients_new['NP3PRSPR'].astype(float)+bs_patients_new['NP3TTAPL'].astype(float)+bs_patients_new['NP3TTAPR'].astype(float)+bs_patients_new['NP3LGAGL'].astype(float)+bs_patients_new['NP3LGAGR'].astype(float)
    return bs_patients_new

In [ ]:
bs_patients_new = add_new_target(bs_patients_new)
bs_hc_new = add_new_target(new_bs_hc)

In [ ]:
bs_patients_new = bs_patients_new[bs_patients_new.PATNO.isin(gene.PATNO)]
print(len(bs_patients_new))
print(len(set(bs_patients_new.PATNO)))


In [ ]:
def events_differ_hc(bs_patients_new,  env ='V04', tar=tar):
        for col in tar:
                bs_patients_new[col]=pd.to_numeric(bs_patients_new[col])
        sp = set(bs_patients_new.PATNO)
        envlist = ['BL', env]
        new_out = []
        for eid in sp:
                pr = bs_patients_new[bs_patients_new.PATNO==eid]
                pr = pr[pr.EVENT_ID.isin(envlist)]
                # here pre should larger than 1
                if len(pr) ==2:
                        current_vist = pr[pr.EVENT_ID=='BL']
                        next_vist = pr[pr.EVENT_ID==env]
                        next_labels = next_vist[tar]
                        # print(next_labels)
                        current_labels = current_vist[tar]
                        # print(current_labels)
                        targets = next_labels.values[0]- current_labels.values[0]
                        new_out.append(targets)
        return new_out

In [ ]:
envss = 'V04'
hc_data = events_differ_hc(bs_hc_new,  env =envss, tar=tar)
new_tar = ["Change_"+t for t in tar]
hc_s =pd.DataFrame(hc_data,columns= new_tar)
print(len(hc_s))
df=hc_s
print(len(df))
df.drop_duplicates(inplace=True)

In [ ]:
bs_patients_new.to_csv(path+'/data/PPMI_pd_M0_36.csv', index=False)

## Data Imputation

In [ ]:
# set rule: next visit must be one year difference
def events_differ(bs_patients_new,  env ='V04', tar=tar, interest_cols = new_feature_cols):
        sp = set(bs_patients_new.PATNO)
        envlist = ['BL', env]
        new_out = []
        for eid in sp:
                pr = bs_patients_new[bs_patients_new.PATNO==eid]
                pr = pr[pr.EVENT_ID.isin(envlist)]
                # here pre should larger than 1
                if len(pr) ==2:
                        current_vist = pr[pr.EVENT_ID=='BL']
                        data = current_vist[interest_cols].values[0]
                        next_vist = pr[pr.EVENT_ID==env]
                        next_labels = next_vist[tar]
                        # print(next_labels)
                        current_labels = current_vist[tar]
                        # print(current_labels)
                        
                        targets = next_labels.values[0]-current_labels.values[0]
                        data= [*data , *targets]
                        new_out.append(data)
        return new_out


In [ ]:

new_tar = ["Change_"+t for t in tar]
new_tar_reg = ["Change_"+t for t in reg_tar]
data = events_differ(bs_patients_new,  env = envss, tar=tar)
pd_patients_all =pd.DataFrame(data,columns= new_feature_cols+new_tar)
pd_patients_all.drop_duplicates(inplace=True)
pd_patients_all.describe().to_csv('results/ppmi_descri_pd_12month.csv')


In [ ]:
pd_patients_all = pd_patients_all.merge(gene, on = 'PATNO', how='inner')
print(len(pd_patients_all))
print(len(set(pd_patients_all.PATNO)))

In [ ]:
## add MRI here
pd_patients_all = pd_patients_all.merge(mri, on = 'PATNO', how='left')
print(len(pd_patients_all))

In [ ]:
pd_patients_all.isna().sum(axis=0).to_csv('results/pd_12month_missing_raw.csv')
hc_s.isna().sum(axis=0).to_csv('results/hc_12month_missing_raw.csv')

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes_list = []
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        axes_list.append(axes[i, j])

for i in range(len(axes_list)):
    ax = axes_list[i]
    hc_s[new_tar_reg[i]].plot(kind='hist', ax =ax, legend=new_tar_reg[i])
fig.savefig('results/hc_12.png', dpi=350)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes_list = []
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        axes_list.append(axes[i, j])

for i in range(len(axes_list)):
    ax = axes_list[i]
    pd_patients_all[new_tar_reg[i]].plot(kind='hist', ax =ax, legend=new_tar_reg[i])
fig.savefig('results/pd_12.png', dpi=350)

Add labels

In [ ]:
def label_func_pos(datas, thr_mean, thr_std, k=1):
    labels =[]
    for data in datas:
    # the smaller the better 
        if data<(thr_mean-k*thr_std):
            label=0
        elif (thr_mean-k*thr_std) <= data <=(thr_mean+k*thr_std):
            label=1
        else:
            label = 2
        labels.append(label)
    return labels

def label_func_neg(datas, thr_mean, thr_std, k=1):
    labels =[]
    for data in datas:
    # the larger the better 
        if data<(thr_mean-k*thr_std):
            label=2
        elif (thr_mean-k*thr_std) <= data <=(thr_mean+k*thr_std):
            label=1
        elif pd.isna(data):
            label = pd.NA
        else:
            label = 0
        labels.append(label)
    return labels

def label_func_hy(data):
    # the smaller the better 
    if data<0:
        label=0
    elif data ==0:
        label = 1
    elif pd.isna(data):
            label = pd.NA
    else:
        label=2
    return label

label_tar = ["Label_"+t for t in tar]

In [ ]:
k=1
pd_patients_all['Label_NP1TOT'] = label_func_pos(pd_patients_all['Change_NP1TOT'], h_thred['Change_NP1TOT']['mean'], h_thred['Change_NP1TOT']['var'],k)
pd_patients_all['Label_NP2PTOT'] = label_func_pos(pd_patients_all['Change_NP2PTOT'], h_thred['Change_NP2PTOT']['mean'], h_thred['Change_NP2PTOT']['var'],k)
pd_patients_all['Label_NP3TOT'] = label_func_pos(pd_patients_all['Change_NP3TOT'], h_thred['Change_NP3TOT']['mean'], h_thred['Change_NP3TOT']['var'],k)
pd_patients_all['Label_NP3_Axial'] = label_func_pos(pd_patients_all['Change_NP3_Axial'], h_thred['Change_NP3_Axial']['mean'], h_thred['Change_NP3_Axial']['var'],k)
pd_patients_all['Label_NP3_Tremor'] = label_func_pos(pd_patients_all['Change_NP3_Tremor'], h_thred['Change_NP3_Tremor']['mean'], h_thred['Change_NP3_Tremor']['var'],k)
pd_patients_all['Label_NP3_Akinetic_Rigid'] = label_func_pos(pd_patients_all['Change_NP3_Akinetic_Rigid'], h_thred['Change_NP3_Akinetic_Rigid']['mean'], h_thred['Change_NP3_Akinetic_Rigid']['var'],k)
pd_patients_all['Label_NPTOT'] =  label_func_pos(pd_patients_all['Change_NPTOT'], h_thred['Change_NPTOT']['mean'], h_thred['Change_NPTOT']['var'],k)
pd_patients_all['Label_ESS_TOT'] =  label_func_pos(pd_patients_all['Change_ESS_TOT'], h_thred['Change_ESS_TOT']['mean'], h_thred['Change_ESS_TOT']['var'],k)
pd_patients_all['Label_MoCA_score'] =  label_func_neg(pd_patients_all['Change_MoCA_score'], h_thred['Change_MoCA_score']['mean'], h_thred['Change_MoCA_score']['var'],k)
pd_patients_all['Label_NHY'] = pd_patients_all['Change_NHY'].apply(label_func_hy)

In [ ]:
pd_patients_all.to_csv(path+'/data/pd_12month_all_mri.csv', index=False)


In [ ]:
def train_test(pd_patients_all, t_label = 'Label_NHY', thred =0.8):

    print('=====================================')
    print('Train Test seperation!!!!')
    pd_patients_all['INFODT']= pd.to_datetime(pd_patients_all['INFODT'])
    pd_patients_all.sort_values(by='INFODT',ascending=True,inplace=True)
    pd_patients_all.reset_index(drop=True,inplace=True)
    m_l = pd_patients_all[t_label].mean()
    postive_case = pd_patients_all[pd_patients_all[t_label]>0]
    negtive_case = pd_patients_all[pd_patients_all[t_label]<=0]
    train_all_idx = list(postive_case[:int(len(postive_case)*thred)].index)+list(negtive_case[:int(len(negtive_case)*thred)].index)
    train_all=pd_patients_all.loc[train_all_idx]
    test = pd_patients_all.drop(train_all_idx)
    train_all=train_all.sample(frac=1)
    test=test.sample(frac=1)
    train_all_idx=train_all.index
    test_idx=test.index
    for label in label_tar: 
        print(label)
        # print("Train:",train_all[label].value_counts()[1]/sum(train_all[label].value_counts())) 
        print("Train:",train_all[label].value_counts())
        # print("test:",test[label].value_counts()[1]/sum(test[label].value_counts()))
        print("test:",test[label].value_counts())
    return train_all, test

In [ ]:
train_all, test= train_test(pd_patients_all_BLV4)